#### Описание проекта

Интернет-магазин товаров для дома «Пока все ещё тут» в срочном порядке ищет аналитиков. Вы поможете нашему магазину стать лучше, а клиентам — обустроить дом своей мечты.

#### Цель проекта:
    
предложить алгоритм разделения всех товаров на основной и дополнительный ассортимент

#### 1. Загрузка и открытие файла ecommerce_dataset.csv, создание датафрейма

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
import pprint
from collections import defaultdict

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('ecommerce_dataset.csv')

Посмотрим, какие у нас есть данные:

In [4]:
df.head(5)

,date,customer_id,order_id,product,quantity,price
0,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0
1,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12,...",1,194.0
2,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0
3,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0
4,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0


In [5]:
df.tail(5)

,date,customer_id,order_id,product,quantity,price
6732,2019103111,73d1cd35-5e5f-4629-8cf2-3fda829d4e58,73156,"Таз пластмассовый 40,0 л пищевой овальный (Иж)...",1,404.0
6733,2019103112,1f0a7f35-7459-4f23-b468-5e45bf481dd1,73158,Сумка-тележка 2-х колесная Gimi Argo синяя,1,1087.0
6734,2019103113,25df96a7-c453-4708-9cea-a3dfc7c342ea,73162,Герань домашняя (пеларгония зональная) цветуща...,1,157.0
6735,2019103114,73d1cd35-5e5f-4629-8cf2-3fda829d4e58,73164,"Таз пластмассовый со стиральной доской (Иж), 1...",1,397.0
6736,2019103116,344aa778-e436-419e-a9c6-9b8f37b7c1df,73137,Сумка-тележка 2-х колесная Gimi Argo синяя,1,1087.0


У нас есть данные за 2018 и 2019 год.

#### 2. Предобработка данных
#### 2.1. Посмотрим типы данных, поменяем тип, если нужно;  посмотрим, есть ли выбросы и дубликаты

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6737 entries, 0 to 6736
Data columns (total 6 columns):
date           6737 non-null int64
customer_id    6737 non-null object
order_id       6737 non-null int64
product        6737 non-null object
quantity       6737 non-null int64
price          6737 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 315.9+ KB


Вывод: 

Размер датафрейма - 6737 строчки, пропусков в данных нет. Формат даты надо поменять на более удобный:

In [7]:
df['date'] = pd.to_datetime((df['date']), format = '%Y%m%d%H')

In [8]:
df.head(5)

,date,customer_id,order_id,product,quantity,price
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12,...",1,194.0
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0


Получили дату в формате год-месяц-день

In [9]:
df.head(5)

,date,customer_id,order_id,product,quantity,price
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12,...",1,194.0
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0


In [10]:
df.duplicated().sum()

0

Явных дубликатов нет.

In [11]:
df.describe()

,order_id,quantity,price
count,6737.000000,6737.000000,6737.000000
mean,43128.948345,2.501559,462.028499
std,27899.414662,15.266478,871.296064
min,12624.000000,1.000000,9.000000
25%,14827.000000,1.000000,101.000000
50%,68503.000000,1.000000,135.000000
75%,70504.000000,1.000000,398.000000
max,73164.000000,1000.000000,14917.000000


Вывод: по результатам применения метода describe() видим, что в среднем в одном заказе - 2,5 товара, минимальное число товаров в заказе - 1, максимальное - 1000 (можем считать выбросом). Средняя стоимость - 462, минимальная - 9, максимальная - 14 917.

##### Посмотрим на данные, которые мы сочли выбросами:

Строка с 1000 товарами:

In [12]:
df[df['quantity'] == 1000]

,date,customer_id,order_id,product,quantity,price
5456,2019-06-18 15:00:00,312e9a3e-5fca-43ff-a6a1-892d2b2d5ba6,71743,"Вантуз с деревянной ручкой d14 см красный, Bur...",1000,675.0


Купили 1000 вантузов.

Посмотрим, есть ли еще какие-то заказы у этого пользователя или какие-то еще действия с этим заказом:

In [13]:
df[df['customer_id'] == '312e9a3e-5fca-43ff-a6a1-892d2b2d5ba6']

,date,customer_id,order_id,product,quantity,price
5456,2019-06-18 15:00:00,312e9a3e-5fca-43ff-a6a1-892d2b2d5ba6,71743,"Вантуз с деревянной ручкой d14 см красный, Bur...",1000,675.0


In [14]:
df[df['order_id'] == 71743]

,date,customer_id,order_id,product,quantity,price
5456,2019-06-18 15:00:00,312e9a3e-5fca-43ff-a6a1-892d2b2d5ba6,71743,"Вантуз с деревянной ручкой d14 см красный, Bur...",1000,675.0


удалим вантузы, так как это явный выброс (ниже в таблице видно, что количество "1000" далеко отстоит от количества товаров в других "больших" заказах:

In [15]:
df = df.drop(np.where(df['order_id'] == 71743)[0])

Строка с ценой 14 917:

In [16]:
df[df['price'] == 14917.000000]

,date,customer_id,order_id,product,quantity,price
5992,2019-07-29 17:00:00,0d87f4ae-465a-4fac-81e6-5d629761783e,72139,Сушилка уличная Leifheit 85210 LINOMATIC V 400...,1,14917.0


Этот товар это уличная сушилка, согласно Гуглу, действительно так стоит, выбросом можно не считать

Посмотрим на всякий случай, еще есть ли еще какие-то заказы у этого клиента или транзакции по этому заказу:

In [17]:
df[df['order_id'] == 72139]

,date,customer_id,order_id,product,quantity,price
5992,2019-07-29 17:00:00,0d87f4ae-465a-4fac-81e6-5d629761783e,72139,Сушилка уличная Leifheit 85210 LINOMATIC V 400...,1,14917.0


In [18]:
df[df['customer_id'] == '0d87f4ae-465a-4fac-81e6-5d629761783e']

,date,customer_id,order_id,product,quantity,price
5992,2019-07-29 17:00:00,0d87f4ae-465a-4fac-81e6-5d629761783e,72139,Сушилка уличная Leifheit 85210 LINOMATIC V 400...,1,14917.0


Посмотрим на заказы с большим количеством товаров:

In [19]:
test = df.sort_values('quantity', ascending=False)

In [20]:
test.head(20)

,date,customer_id,order_id,product,quantity,price
5071,2019-06-11 07:00:00,146cd9bf-a95c-4afb-915b-5f6684b17444,71668,Вешалки мягкие для деликатных вещей 3 шт шоколад,334,148.0
3961,2019-05-20 21:00:00,5d189e88-d4d6-4eac-ab43-fa65a3c4d106,71478,Муляж ЯБЛОКО 9 см красное,300,51.0
1158,2018-12-10 14:00:00,a984c5b7-ff7e-4647-b84e-ef0b85a2762d,69289,"Ручка-скоба РС-100 белая *Трибатрон*, 1108035",200,29.0
568,2018-11-01 08:00:00,aa42dc38-780f-4b50-9a65-83b6fa64e766,68815,Муляж ЯБЛОКО 9 см красное,170,51.0
267,2018-10-16 08:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,Пружина дверная 240 мм оцинкованная (Д-19 мм) ...,150,38.0
211,2018-10-11 14:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,"Крепеж для пружины дверной, 1107055",150,19.0
212,2018-10-11 14:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,Пружина дверная 240 мм оцинкованная (Д-19 мм) ...,150,38.0
266,2018-10-16 08:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,"Крепеж для пружины дверной, 1107055",150,19.0
2431,2019-03-23 10:00:00,685d3d84-aebb-485b-8e59-344b3df8b3d3,70841,Плечики пластмассовые Размер 52 - 54 Тула 1205158,150,20.0
586,2018-11-02 11:00:00,0c5aaa88-e346-4f87-8f7a-ad8cbc04e965,68831,Муляж ЯБЛОКО 9 см красное,140,59.0


Выводы: 

1. Оказалось много заказов с огромным количеством позиций в заказе (334, 300, 200), но при этом из таблицы describe было видно, что 75% заказов содержат по 1 товару. Возможно, это заказы оптовиков или перекупщиков: товаров куплено много, но они при этом все относительно дешевые, например, 150 крепежей для двери, каждый по 19 денежных единиц (может быть, спеццены для опта?.


2. Кроме этого, есть, например, заказ №68611, у которого один и тот же заказчик (customer_id одинаковый для всех строк), но заказ с этим номером фиксируется несколько раз с разным временем, c интервалом в несколько дней: то есть, тогда это уже время поступления заказа/ обработки/ отгрузки и подобного, мы точно не знаем.

In [21]:
df[df['order_id'] == 68611]

,date,customer_id,order_id,product,quantity,price
211,2018-10-11 14:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,"Крепеж для пружины дверной, 1107055",150,19.0
212,2018-10-11 14:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,Пружина дверная 240 мм оцинкованная (Д-19 мм) ...,150,38.0
266,2018-10-16 08:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,"Крепеж для пружины дверной, 1107055",150,19.0
267,2018-10-16 08:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,Пружина дверная 240 мм оцинкованная (Д-19 мм) ...,150,38.0


4 транзакции с этим номером заказа в датасете

Поэтому посмотрим на количество уникальных значений в столбце ['order_id']:

In [22]:
len(df['order_id'].unique())

2783

2783 уникальных заказа из 6737 записанных в датасет строк, то есть у нас, по сути, датасет - список всех логов/  "транзакций", совершенных с этими заказами; строки с одними и теми же товарами, соответственно, могут повторяться несколько раз. 

Выводы по разделу с предполагаемыми "выбросами": 

1. Удалила явный выброс - строку с 1000 вантузами в заказе - так как она является крайним значением по describe() и выделяется на фоне остальных больших значений, следующее после нее  - 334 вешалки в заказе №71668.


2. Я бы не стала удалять остальные строки с большим количеством товаров, так как они составляют только до 25% от всего датасета. 75% заказов содержат 1 товар.


3. Повторяющиеся строки в датасете тоже не стала бы удалять, потому что, мы не знаем, что это за клиенты  и для каких целей были закупки: по первым 20 строкам test'a ощущение, что это большие закупки для ремонтных/ строительных бригад (дверные пружины, стяжки) и для модных съемок (вешалки, плечики, муляжи фруктов и т.д.). 


На цель проекта - выделить категории ассортимента - повторяющиеся строки не влияют напрямую, то есть логика разделения сначала на смысловые категории (растения, товары для дома и тд), а потом на основной и дополнительный ассортимент остается прежней, но могут повлиять на соотношения долей категорий ко всему ассортименту.


При необходимости, данные можно очистить/подготовить для лемматизации или другого подобного метода. 

#### 3. Посмотрим данные - количество уникальных товаров/ пользователей, товары, которые чаще всего покупали и т.д. 

Посмотрим, сколько уникальных пользователей:

In [23]:
df['customer_id'].nunique()

2450

2 450 уникальных пользователей

Посмотрим, сколько уникальных продуктов:

In [24]:
df['product'].nunique()

2342

2 342 уникальных продукта

Посмотрим, какие товары чаще и реже всего встречаются в наборе данных:

In [25]:
print(df['product'].value_counts().head(20)) 

Рассада Кабачка сорт Зебра кассета 6 шт E6                              79
Рассада Кабачка сорт Золотинка кассета 6 шт E6                          78
Рассада Кабачка сорт Белогор кассета 6 шт E6                            77
Рассада Огурца сорт Каролина кассета 6 шт E6                            76
Рассада Огурец Фараон ячейки по 6 шт Е6                                 76
Пеларгония розебудная Red Pandora укорененный черенок                   76
Рассада Кабачка сорт Аэронавт, кассета по 6шт                           76
Рассада Кабачка сорт Горный, кассета по 6шт                             76
Тагетис крупноцветковый рассада однолетних цветов в кассете по 10 шт    76
Рассада Патиссона сорт НЛО Белый, кассета по 6шт                        75
Рассада Кабачка сорт Сосновский, кассета по 6шт                         75
Рассада Патиссона сорт Диск, кассета по 6шт                             74
Рассада Огурец Кристина ячейки по 6 шт Е6                               74
Сумка-тележка 2-х колесна

Видим, что очень часто встречается "рассада", "сумка-тележка", и различные сорта цветов: "пеларгония", "роза" и т.д. + по первому названию товара в принципе понятно, что он собой представляет: цветы/ товары для дома и т.д.

In [26]:
print(df['product'].value_counts().tail(15)) 

Сверло-фреза, 6 мм, нитрид титана *888* 6862060, 0512262                              1
Скатерть Dasch Донна Роза из ПВХ 0018-8 Букет овальная 132х178 см 5979                1
Набор вешалок пластмассовых Rozenbal R181900                                          1
Термостакан WEBBER BE-6030 КОФЕ 450 мл                                                1
Паста для полировки "ГОИ" 25 гр                                                       1
Петрушка листовая Универсал в кассете 4 штуки среднеспелая                            1
Сумка-тележка 2-х колесная Gimi Argo фуксия                                           1
Комнатное растение в горшке Плектрантус подвесной горшок, d16, h30                    1
Искусственный цветок Ветка петунии пластиковая лиловая                                1
Роза патио белая d-12 см h-30                                                         1
Сушилка для белья на стену Leifheit Telegant 100, белая, 83100                        1
Котел алюминиевый 70,0 л без кры

#### 4. Создадим категории товаров, выделим основные и дополнительные

##### Преобразуем товары, которые у нас есть в список и сделаем лемматизацию:

In [27]:
unique_products = df['product'].unique().tolist()

In [28]:
m = Mystem()
string = '; '.join(unique_products)

In [29]:
lemmas = m.lemmatize(string)
pprint.pprint(Counter(lemmas))

Counter({' ': 15369,
         '; ': 2323,
         ', ': 1064,
         'см': 1062,
         '-': 980,
         'для': 547,
         ',': 295,
         'в': 274,
         'л': 253,
         'с': 253,
         'd': 252,
         'рассада': 221,
         '5': 190,
         'белый': 173,
         ' (': 172,
         'сорт': 171,
         'кассета': 170,
         'шт': 169,
         '0': 164,
         '12': 148,
         'белье': 144,
         'красный': 143,
         ') ': 140,
         'доска': 132,
         '/': 132,
         'горшок': 130,
         '9': 123,
         'сушилка': 120,
         'цветок': 118,
         'объем': 114,
         'гладильный': 113,
         '2': 113,
         '6': 110,
         'тележка': 108,
         '1': 108,
         '. ': 106,
         'по': 101,
         '4': 101,
         '10': 100,
         'томат': 100,
         'сумка': 98,
         'диам': 95,
         'помидор': 93,
         '*': 90,
         '№': 88,
         'пеларгония': 86,
         'розовый': 8

Выделилось много цветов и товаров для дома.

Так как выше мы уже решили, что по первому слову названия товара понятно, что это за товар, то разобьем названия всех товаров и выделим первое слово в отдельный столбец:

In [30]:
df['product_first_word'] = df['product'].apply(lambda x: x.split()[:1]).apply(lambda x: ' '.join(x))
df.head(5)

,date,customer_id,order_id,product,quantity,price,product_first_word
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0,Комнатное
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12,...",1,194.0,Комнатное
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0,Радермахера
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0,Хризолидокарпус
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0,Циперус


Посмотрим на получившееся множество товаров, которые затем объединим по категориям.

In [31]:
goods_list = df['product_first_word'].unique().tolist()
df['product_first_word'].unique()

array(['Комнатное', 'Радермахера', 'Хризолидокарпус', 'Циперус',
       'Шеффлера', 'Юкка', 'Настенная', 'Таз', 'Чехол', 'Вешалка',
       'Сушилка', 'Крючок', 'Корзина', 'Мини-сковорода', 'Сковорода',
       'Стеклянная', 'Доска', 'Сумка-тележка', 'Многолетнее', 'Рассада',
       'Муляж', 'Щетка-сметка', 'Сумка', 'Тарелка', 'Вилка', 'Ложка',
       'Нож', 'Герань', 'Ерш', 'Пеларгония', 'Котел', 'Суккулент',
       'Эхеверия', 'Набор', 'Коврик', 'Подкладка', 'Дозатор', 'Ёрш',
       'Эвкалипт', 'Цветок', 'Мусорный', 'Плед', 'Скатерть', 'Кувшин',
       'Адиантум', 'Ковер', 'Перчатки', 'Стремянка', 'Гладильная',
       'Лестница-стремянка', 'Толкушка', 'Тележка', 'Готовая',
       'Искусственный', 'Простыня', 'Чайник', 'Штора', 'Банка', 'Стяжка',
       'Комплект', 'Кампанула', 'Декоративная', 'Противень', 'Калатея',
       'Нолина', 'Плечики', 'Подушка', 'Полотенце', 'Хлорофитум',
       'Цикламен', 'Крепеж', 'Пружина', 'Глоксиния', 'Азалия',
       'Гортензия', 'Хризантема', 'Петля', 

In [32]:
goods_list

['Комнатное',
 'Радермахера',
 'Хризолидокарпус',
 'Циперус',
 'Шеффлера',
 'Юкка',
 'Настенная',
 'Таз',
 'Чехол',
 'Вешалка',
 'Сушилка',
 'Крючок',
 'Корзина',
 'Мини-сковорода',
 'Сковорода',
 'Стеклянная',
 'Доска',
 'Сумка-тележка',
 'Многолетнее',
 'Рассада',
 'Муляж',
 'Щетка-сметка',
 'Сумка',
 'Тарелка',
 'Вилка',
 'Ложка',
 'Нож',
 'Герань',
 'Ерш',
 'Пеларгония',
 'Котел',
 'Суккулент',
 'Эхеверия',
 'Набор',
 'Коврик',
 'Подкладка',
 'Дозатор',
 'Ёрш',
 'Эвкалипт',
 'Цветок',
 'Мусорный',
 'Плед',
 'Скатерть',
 'Кувшин',
 'Адиантум',
 'Ковер',
 'Перчатки',
 'Стремянка',
 'Гладильная',
 'Лестница-стремянка',
 'Толкушка',
 'Тележка',
 'Готовая',
 'Искусственный',
 'Простыня',
 'Чайник',
 'Штора',
 'Банка',
 'Стяжка',
 'Комплект',
 'Кампанула',
 'Декоративная',
 'Противень',
 'Калатея',
 'Нолина',
 'Плечики',
 'Подушка',
 'Полотенце',
 'Хлорофитум',
 'Цикламен',
 'Крепеж',
 'Пружина',
 'Глоксиния',
 'Азалия',
 'Гортензия',
 'Хризантема',
 'Петля',
 'Салфетка',
 'Вкладыши',
 '

Так как у нас много наименований рассады, предлагаю выделить ее в отдельную категорию "Рассада" + выделить еще категории:

- "Растения", куда войдут цветы/ овощи/ фрукты, 

- "Товары для дома", "Садовый инвентарь", "Инструменты и фурнитура"

- "Другое"

In [33]:
from collections import defaultdict
category_dict = defaultdict(list)

In [34]:
category_dict = {'Растения': ['Комнатное', 'Радермахера', 'Хризолидокарпус', 'Циперус','Шеффлера', 'Юкка', 'Многолетнее', 'Герань', 'Пеларгония', 'Суккулент', 'Эхеверия', 'Эвкалипт', 'Цветок', 'Адиантум', 'Искусственный', 'Кампанула','Калатея','Нолина', 'Хлорофитум', 'Цикламен', 'Глоксиния', 'Азалия', 'Гортензия', 'Хризантема', 'Мирт', 'Цветущее', 'Кориандр', 'Мускари', 'Пиретрум',  'Агератум', 'Колеус', 'Тагетис', 'Цинерания', 'Муррайя','Портулак', 'Астра', 'Аргирантерум', 'Левкой', 'Вероника','Целозия', 'Арбуз', 'Баклажан', 'Клубника', 'Патиссон', 'Маттиола','Сальвия', 'Лук', 'Джункус', 'Драцена', 'Фатсия', 'Хоста', 'Гиностемма', 'Ель', 'Кипарисовик', 'Аптения', 'Антуриум', 'Молодило', 'Фиалка', 'Гербера', 'Душица', 'Мелисса', 'Роза', 'Бегония', 'Розмарин', 'Бальзамин', 'Крассула', 'Артемизия', 'Иссоп','Сантолина', 'Аспарагус', 'Афеляндра', 'Альбука','Лаванда', 'Мята', 'Каланхое', 'Примула', 'Алиссум', 'Гвоздика', 'Годеция', 'Календула','Капуста', 'Кореопсис', 'Лапчатка', 'Львиный', 'Флокс','Цинерария', 'Эшшольция', 'Диффенбахия','Фаленопсис', 'Бархатцы', 'Космея', 'Морковь', 'Настурция','Огурец', 'Петуния', 'Анемона', 'Буддлея', 'Валериана', 'Змееголовник', 'Любисток', 'Петрушка', 'Сельдерей', 'Шалфей', 'Дендробиум','Калоцефалус', 'Каллуна', 'Капсикум', 'Мимоза','Соланум', 'Фуксия', 'Цитрофортунелла','Пеперомия', 'Дыня', 'Циния', 'Синнингия', 'Гардения', 'Лобелия', 'Лаватера', 'Вигна','Кабачок', 'Тыква', 'Хамедорея', 'Зверобой', 'Чабер', 'Бакопа', 'Ранункулус', 'Лилейник', 'Папоротник', 'Платикодон', 'Энотера', 'Калибрахоа', 'Эпипремнум', 'Газания', 'Овсянница', 'Смолевка', 'Цинния', 'Лавр', 'Вербейник', 'Виола', 'Колокольчик','Котовник', 'Эхинацея', 'Ясколка', 'Подсолнечник', 'Гайлардия', 'Монарда', 'Нивянник', 'Рудбекия', 'Табак','Девичий', 'Сциндапсус', 'Пряные', 'Бузульник', 'Камнеломка', 'Солидаго','Физостегия', 'Бадан', 'Лен', 'Искусственная', 'Кодонанта',       'Цикламен,', 'Гимнокалициум', 'Кофе', 'Пуансетия', 'Пуансеттия','Эхинокактус', 'Скиммия', 'Фиттония', 'Аквилегия', 'Базилик', 'Горох', 'Земляника', 'Незабудка', 'Седум', 'Томат', 'Укроп', 'Нефролепис', 'Калла', 'Декабрист', 'Тюльпан', 'Гиацинт', 'Вербена', 'Однолетнее', 'Георгина', 'Тимьян', 'Алоэ', 'Спатифиллум', 'Фикус', 'Крокусы', 'Гипсофила', 'Барвинок', 'Осина', 'Лантана',  'Мединилла', 'Импатиенс', 'Клен', 'Пахира', 'Салат', 'Замиокулькас', 'Литопс','Амариллис'], 'Рассада': ['Рассада'], 'Товары для дома': ['Настенная', 'Таз', 'Чехол', 'Вешалка', 'Сушилка', 'Крючок', 'Корзина', 'Мини-сковорода', 'Сковорода', 'Стеклянная', 'Доска', 'Сумка-тележка', 'Щетка-сметка', 'Сумка', 'Тарелка', 'Вилка', 'Ложка', 'Ерш', 'Набор', 'Коврик', 'Ёрш', 'Нож', 'Мусорный', 'Плед', 'Скатерть', 'Кувшин', 'Ковер', 'Стремянка', 'Гладильная', 'Лестница-стремянка', 'Толкушка', 'Тележка', 'Простыня', 'Чайник', 'Штора', 'Банка', 'Комплект', 'Противень', 'Плечики', 'Подушка', 'Полотенце', 'Петля', 'Салфетка', 'Вкладыши', 'Миксер', 'Кофр', 'Стиральный', 'Чистящий', 'Tepмокружка', 'Щетка', 'Стремянки', 'Щетка-утюжок', 'Светильник', 'Жидкое', 'Ковёр', 'Пена', 'Вакуумный', 'Тряпка', 'Половник', 'Двуспальное', 'Кипятильник', 'Сверло-фреза,','Контейнер', 'Зубная', 'Карниз', 'Веник', 'Швабра', 'Ящик', 'Подголовник', 'Ведро', 'Короб', 'Коробка', 'Наматрасник','Наматрицник-чехол', 'Насадка', 'Ножеточка', 'Вешалка-плечики',  'Настольная', 'Сиденье', 'Масленка', 'Утюг', 'Корыто', 'Холодная', 'Сервировочная', 'Салатник', 'Шнур', 'Урна','Чайный', 'Мыло', 'Рыбочистка', 'Овощеварка','Сметка', 'Халат','Измерительный', 'Линейка,', 'Терка', 'Кружка', 'Лопатка', 'Салфетница', 'Стакан', 'Столовая', 'Чайная', 'Жестяная', 'Рассекатель', 'Ополаскиватель', 'Сетка', 'Покрывало', 'Весы', 'Овощечистка', 'Средство', 'Бензин', 'Паста', 'Подрукавник', 'Вешалка-стойка', 'Держатель', 'Пробка', 'Измельчитель', 'Лоток', 'Вешалки', 'Губка', 'Фен', 'Ванна','Корзинка', 'Ваза', 'Антижир', 'Универсальное', 'Мантоварка', 'Ролик', 'Сменный', 'Термометр', 'Шило', 'Ковш', 'Кухонные', 'Кисточка', 'Скребок', 'Комод', 'Картофелемялка', 'Ароматизированное', 'Бальзам', 'Подставка', 'Вешалка-сушилка', 'Миска', 'Одеяло', 'Совок', 'Хлебница', 'Насадка-отжим', 'Наматрацник', 'Автоматическая', 'Тряпкодержатель', 'Скалка','Рукав', 'Венчик', 'Сито', 'Бак','Концентрат', 'Тортница', 'Соковарка', 'Пьезозажигалка', 'Соковыжималка', 'Решетка','Запасная', 'Щётка', 'Универсальный', 'Блюдо', 'Полка', 'Окномойка', 'Урна-пепельница', 'Сменная', 'Ткань', 'Нетканые', 'Пылесос','Насадка-моп', 'Пресс', 'Термостакан', 'Новогоднее','Мыльница', 'Защитный', 'Многофункциональный', 'Стремянка-табурет', 'Кольца', 'Вантуз', 'Термокружка', 'Фоторамка','Этажерка', 'Термос', 'Кондиционер', 'Фарфоровая','Электроштопор', 'Простынь', 'Столовый', 'Блюдце', 'Махровый',  'Мыло-скраб', 'Подвесное', 'Бельевые', 'Полки', 'Бульонница', 'Ключница', 'Разделочная','Мантоварка-пароварка', 'Складная', 'Емкость', 'Гипоаллергенный', 'Вешалка-перекладина','Модульная', 'Лестница', 'Просеиватель', 'Кастрюля', 'Махровое','Форма', 'Орехоколка',  'Увлажняющая',
'Кухонное', 'Подарочный', 'Наволочка', 'Обувница-3', 'Пододеяльник', 'Антинакипин', 'Сотейник','Стеллаж', 'Бидон', 'Складной', 'Прищепки', 'Шприц', 'Крышка', 'Покрытие', 'Мерный', 'Веревка', 'Мешок', 'Отбеливатель', 'Сахарница'], 'Садовый инвентарь': ['Муляж', 'Котел', 'Подкладка', 'Дозатор', 'Перчатки', 'Декоративная', 'Защитная'], 'Инструменты и фурнитура': ['Пружина', 'Петля-стрела', 'Уголок', 'Ручка-скоба', 'Фал', 'Штангенциркуль', 'Штанга', 'Сверло', 'Фиксатор-шар', 'Шпагат', 'Отделитель', 'Шпингалет', 'Завертка', 'Ручка', 'Стяжка', 'Стяжки', 'Крепеж']}

С помощью функции присвоим товарам категории и создадим новый столбец с получившейся категорией для каждого товара:

In [35]:
def category_func(name):
    for key, value in category_dict.items():
        for word in value:
            if name.find(word) > -1:
                return key
    return 'Другое'

In [36]:
df['category'] = df['product'].apply(category_func)

Получили, что самые большие по количеству товаров категории это "Растения" и "Товары для дома", но нам еще нужно определить, что является основным, а что дополнительным ассортиментом магазина. 
Основываясь на размерах созданных товарных категорий, мне кажется, что логично будет объединить категории "Растения", "Рассада"  и "Садовый инвентарь", они все о садоводстве, и определить их основным ассортиментом; а "Товары для дома", "Другое" и          "Инструменты и фурнитура" сделать дополнительным ассортиментом. Тогда мы получим 4180 товаров в основном ассортименте магазина, и 2556 товаров - в дополнительном.

In [37]:
df['category'].value_counts(ascending = False)

Растения                   3211
Товары для дома            2364
Рассада                     755
Садовый инвентарь           214
Другое                      157
Инструменты и фурнитура      35
Name: category, dtype: int64

In [38]:
df.head(5)

,date,customer_id,order_id,product,quantity,price,product_first_word,category
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0,Комнатное,Растения
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12,...",1,194.0,Комнатное,Растения
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0,Радермахера,Растения
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0,Хризолидокарпус,Растения
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0,Циперус,Растения


#### Разделим на основной и дополнительный ассортимент:

In [39]:
def main_range(category):
    if category == 'Растения':
        return 'Основной ассортимент'
    if category == 'Рассада':
        return 'Основной ассортимент'
    if category == 'Садовый инвентарь':
        return 'Основной ассортимент'
    return 'Дополнительный ассортимент' 

In [40]:
df['range'] = df['category'].apply(main_range)

In [41]:
df['range'].value_counts(ascending = False)

Основной ассортимент          4180
Дополнительный ассортимент    2556
Name: range, dtype: int64

In [42]:
df.head(5)

,date,customer_id,order_id,product,quantity,price,product_first_word,category,range
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0,Комнатное,Растения,Основной ассортимент
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12,...",1,194.0,Комнатное,Растения,Основной ассортимент
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0,Радермахера,Растения,Основной ассортимент
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0,Хризолидокарпус,Растения,Основной ассортимент
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0,Циперус,Растения,Основной ассортимент


In [43]:
df.groupby(['customer_id', 'order_id', 'product']).count().sort_values('date', ascending=False).head()

date  \
customer_id                          order_id product                                                  
1d6c8c1f-a799-4418-9af2-1ded47d7a85c 14833    Рассада Огурца сорт Каролина кассета 6 шт E6        74   
                                              Рассада Кабачка сорт Аэронавт, кассета по 6шт       74   
                                              Рассада Патиссона сорт НЛО Белый, кассета по 6шт    74   
                                              Рассада Патиссона сорт Диск, кассета по 6шт         74   
                                              Рассада Огурец Фараон ячейки по 6 шт Е6             74   

                                                                                                quantity  \
customer_id                          order_id product                                                      
1d6c8c1f-a799-4418-9af2-1ded47d7a85c 14833    Рассада Огурца сорт Каролина кассета 6 шт E6            74   
                                              Рассада Кабачка сорт Аэронавт, кассета по 6шт           74   
                                              Рассада Патиссона сорт НЛО Белый, кассета по 6шт        74   
                                              Рассада Патиссона сорт Диск, кассета по 6шт             74   
                                              Рассада Огурец Фараон ячейки по 6 шт Е6                 74   

                                                                                                price  \
customer_id                          order_id product                                                   
1d6c8c1f-a799-4418-9af2-1ded47d7a85c 14833    Рассада Огурца сорт Каролина кассета 6 шт E6         74   
                                              Рассада Кабачка сорт Аэронавт, кассета по 6шт        74   
                                              Рассада Патиссона сорт НЛО Белый, кассета по 6шт     74   
                                              Рассада Патиссона сорт Диск, кассета по 6шт          74   
                                              Рассада Огурец Фараон ячейки по 6 шт Е6              74   

                                                                                                product_first_word  \
customer_id                          order_id product                                                                
1d6c8c1f-a799-4418-9af2-1ded47d7a85c 14833    Рассада Огурца сорт Каролина кассета 6 шт E6                      74   
                                              Рассада Кабачка сорт Аэронавт, кассета по 6шт                     74   
                                              Рассада Патиссона сорт НЛО Белый, кассета по 6шт                  74   
                                              Рассада Патиссона сорт Диск, кассета по 6шт                       74   
                                              Рассада Огурец Фараон ячейки по 6 шт Е6                           74   

                                                                                                category  \
customer_id                          order_id product                                                      
1d6c8c1f-a799-4418-9af2-1ded47d7a85c 14833    Рассада Огурца сорт Каролина кассета 6 шт E6            74   
                                              Рассада Кабачка сорт Аэронавт, кассета по 6шт           74   
                                              Рассада Патиссона сорт НЛО Белый, кассета по 6шт        74   
                                              Рассада Патиссона сорт Диск, кассета по 6шт             74   
                                              Рассада Огурец Фараон ячейки по 6 шт Е6                 74   

                                                                                                range  
customer_id                          order_id product                                                  
1d6c8c1f-a799-4418-9af2-1ded47d7a85c 14833    Рассада Огурца сорт Каролина кассета

In [44]:
df1 = df[df['order_id'] == 14833]

In [45]:
df1.head(10)

,date,customer_id,order_id,product,quantity,price,product_first_word,category,range
4157,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,"Рассада Кабачка сорт Аэронавт, кассета по 6шт",1,120.0,Рассада,Рассада,Основной ассортимент
4158,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,Рассада Кабачка сорт Белогор кассета 6 шт E6,1,120.0,Рассада,Рассада,Основной ассортимент
4159,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,"Рассада Кабачка сорт Горный, кассета по 6шт",1,120.0,Рассада,Рассада,Основной ассортимент
4160,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,Рассада Кабачка сорт Зебра кассета 6 шт E6,1,120.0,Рассада,Рассада,Основной ассортимент
4161,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,Рассада Кабачка сорт Золотинка кассета 6 шт E6,1,120.0,Рассада,Рассада,Основной ассортимент
4162,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,"Рассада Кабачка сорт Сосновский, кассета по 6шт",1,120.0,Рассада,Рассада,Основной ассортимент
4163,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,Рассада Огурец Кристина ячейки по 6 шт Е6,1,120.0,Рассада,Растения,Основной ассортимент
4164,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,Рассада Огурец Фараон ячейки по 6 шт Е6,1,120.0,Рассада,Растения,Основной ассортимент
4165,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,Рассада Огурца сорт Каролина кассета 6 шт E6,1,120.0,Рассада,Рассада,Основной ассортимент
4166,2019-05-31 12:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,"Рассада Патиссона сорт Диск, кассета по 6шт",1,120.0,Рассада,Растения,Основной ассортимент


In [46]:
df1.groupby(['date']).count().sort_values('product', ascending=False)

,customer_id,order_id,product,quantity,price,product_first_word,category,range
date,,,,,,,,
2019-05-31 12:00:00,12,12,12,12,12,12,12,12
2019-06-13 20:00:00,12,12,12,12,12,12,12,12
2019-06-13 10:00:00,12,12,12,12,12,12,12,12
2019-06-13 08:00:00,12,12,12,12,12,12,12,12
2019-06-12 19:00:00,12,12,12,12,12,12,12,12
...,...,...,...,...,...,...,...,...
2019-06-05 22:00:00,12,12,12,12,12,12,12,12
2019-06-05 20:00:00,12,12,12,12,12,12,12,12
2019-06-05 14:00:00,12,12,12,12,12,12,12,12


Вывод: заказ №14833, клиент один и тот же, 12 товаров в заказе, на протяжении времени с 31 мая 2019 по 22 июня 2019 у заказа менялся статус.  Количество транзакций с одним и тем же заказом напрямую не влияет на цель проекта - разделить ассортимент на основной и дополнительный, так что строки транзакций удалять не буду.

#### 5. Сформулируем нулевые и альтернативные гипотезы

Нулевая гипотеза: Средний показатель выручки по товарам основного ассортимента равен среднему показателю выручки по товарам дополнительного ассортимента

Альтернативная гипотеза: Средний показатель выручки по товарам основного ассортимента не равен среднему показателю выручки по товарам дополнительного ассортимента

Найдем выручку:

In [47]:
df['revenue']=df['quantity']*df['price']

In [48]:
df.head(5)

,date,customer_id,order_id,product,quantity,price,product_first_word,category,range,revenue
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0,Комнатное,Растения,Основной ассортимент,142.0
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12,...",1,194.0,Комнатное,Растения,Основной ассортимент,194.0
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0,Радермахера,Растения,Основной ассортимент,112.0
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0,Хризолидокарпус,Растения,Основной ассортимент,179.0
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0,Циперус,Растения,Основной ассортимент,112.0


Выделим выручку основного и дополнительного ассортимента:

In [49]:
main_range = df.query('range == "Основной ассортимент"')

In [50]:
rest_range = df.query('range == "Дополнительный ассортимент"')

In [51]:
sample_1 = main_range['revenue']

In [52]:
sample_2 = rest_range['revenue']

In [53]:
alpha = .05
results = st.ttest_ind(sample_1, sample_2)

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу') 

p-значение: 1.4351653937046297e-191
Отвергаем нулевую гипотезу


Вывод: отвергаем нулевую гипотезу, значит, средний показатель выручки по товарам основного ассортимента не равен среднему показателю выручки по товарам дополнительного ассортимента

#### 6. Выводы

В рамках этого проекта мы анализировали интернет-магазин товаров для дома. Целью проекта было предложить алгоритм разделения ассортимента на основной и дополнительный. 

В процессе работы мы предобработали данные, поменяли даты на более удобный формат, изучили выбросы.

Выяснили, что в датасете есть строки с записями всех изменений статуса по одному и тому же заказу, оставили эти строки, как есть.

Далее мы сделали лемматизацию и разбивку товаров по первому слову в названии и сделали вывод, что наиболее часто повторяющиеся в названии товара слова: "рассада", "сумка-тележка", "пеларгония" и другие, - позволяют идентифицировать, к какой группе товаров они относятся и могут стать основанием для разделения ассортимента на категории.  

Сначала для удобства мы выделили и присвоили 6 категорий товаров: "Растения", "Товары для дома", "Рассада", "Садовый инвентарь", "Другое", "Инструменты и фурнитура". "Рассаду" выделили отдельно, так как на наш взгляд она составляет большую долю всего ассортимента (11%) и отличается от "Растений". Далее, не смотря на то, что наш интернет-магазин в описании проекта назван "магазином товаров для дома" по итогам категоризации у нас получилось, что бОльшую долю ассортимента занимают "растения", "рассада" и "садовый инвентарь" (62% ассортимента), так что выглядело логичным присвоить этим категориям статус "основного ассортимента", а "другое", "товары для дома", "инструменты и фурнитура" отнести к дополнительному ассортименту. Разделения на категории мы делали с помощью создания функции и новых столбцов с категориями. 

Далее проверили гипотезу о равенстве средних показателей выручки основного и дополнительного ассортимента при уровне значимости 0.05, применив критерий Стьюдента. В результате нулевая гипотеза отвергнута - средний показатель выручки по товарам основного ассортимента не равен среднему показателю выручки по товарам дополнительного ассортимента.